<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Loading data</a></span><ul class="toc-item"><li><span><a href="#Sanity-check" data-toc-modified-id="Sanity-check-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Sanity check</a></span></li><li><span><a href="#Loading-all-reprots" data-toc-modified-id="Loading-all-reprots-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Loading all reprots</a></span></li><li><span><a href="#Narrow-down-based-on-country-and-qualification" data-toc-modified-id="Narrow-down-based-on-country-and-qualification-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Narrow down based on country and qualification</a></span></li></ul></li><li><span><a href="#Generate-data-for-all-patients" data-toc-modified-id="Generate-data-for-all-patients-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generate data for all patients</a></span><ul class="toc-item"><li><span><a href="#Generate-data" data-toc-modified-id="Generate-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Generate data</a></span><ul class="toc-item"><li><span><a href="#Sanity-check" data-toc-modified-id="Sanity-check-2.1.1"><span class="toc-item-num">2.1.1&nbsp;&nbsp;</span>Sanity check</a></span></li></ul></li><li><span><a href="#Population-distribution" data-toc-modified-id="Population-distribution-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Population distribution</a></span></li></ul></li><li><span><a href="#Conditioned-on-Gender" data-toc-modified-id="Conditioned-on-Gender-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Conditioned on Gender</a></span><ul class="toc-item"><li><span><a href="#Male" data-toc-modified-id="Male-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Male</a></span></li><li><span><a href="#Female" data-toc-modified-id="Female-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Female</a></span></li></ul></li><li><span><a href="#Conditioned-on-Age" data-toc-modified-id="Conditioned-on-Age-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Conditioned on Age</a></span><ul class="toc-item"><li><span><a href="#Bin-age-into-groups" data-toc-modified-id="Bin-age-into-groups-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Bin age into groups</a></span></li><li><span><a href="#Three-age-groups" data-toc-modified-id="Three-age-groups-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Three age groups</a></span></li></ul></li></ul></div>

# Loading data

We investigate period from 03-11 to 09-30 from 2013 to 2020. If anyone want to analyze different time period, just replace the start or end time. For example, replace '09-30' by '12-31' to study the period from March 11 to December 31. 

In [4]:
import itertools
from tqdm import tqdm
import pandas as pd
import pickle
import numpy as np
from collections import Counter
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# %matplotlib notebook
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')

# load the dictionaries for drugs, AE
se_dic = pickle.load(open('../Data/curated/AE_dic.pk', 'rb'))
drug_dic = pickle.load(open('../Data/curated/drug_mapping.pk', 'rb'))

# In this MeDRA_dic, key is string of PT_name, value is a list:
# [PT, PT_name, HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr]
meddra_pd_all = pickle.load(open('../Data/curated/AE_mapping.pk', 'rb'))

In [5]:
def chi_sq(A, B,C,D):
    A, B, C,D = A.astype(float), B.astype(float), C.astype(float), D.astype(float)
    numerator =(A*D-B*C)**2
    denominator = (A+B)*(C+D)*(A+C)*(B+D)
    numerator = numerator/denominator
    numerator_ = numerator*(A+B+C+D)
    return numerator_

def lower_CI(A, B,C,D, ROR):
    A, B, C,D, ROR = A.astype(float), B.astype(float), C.astype(float), D.astype(float), ROR.astype(float)
    s = np.sqrt(1/A+ 1/B + 1/C +1/D)
    CI_low = np.e**(np.log(ROR)-1.96*s)
#     CI_high = e**(np.log(ROR)+1.96*s)
    return CI_low

## Sanity check

In [6]:
meddra_pd_all.head(1)

,PT,PT_name,HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr
0,10000002,11-beta-hydroxylase deficiency,10021608,inborn errors of steroid synthesis,10027424,metabolic and nutritional disorders congenital,10010331,"congenital, familial and genetic disorders",Cong


In [7]:
meddra_pd_all[meddra_pd_all.PT=='10018358']

,PT,PT_name,HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr
0,10018358,glomerular filtration rate decreased,10038454,renal function analyses,10038362,renal and urinary tract investigations and uri...,10022891,investigations,Inv


In [8]:
# drug dictionary to df
drug_dic_pd = pd.DataFrame(drug_dic)
drug_dic_df = drug_dic_pd.T

drug_dic_df.columns=['drugbank_ID', 'code']
drug_dic_df.head()

,drugbank_ID,code
etanercept,DB00005,4
zoledronic acid,DB00399,334
atorvastatin,DB01076,931
metformin,DB00331,274
rosiglitazone,DB00412,343


In [9]:
drug_dic_df[drug_dic_df.code == 931]
# drug_dic['remdesivir']

# Make a drug_code_dic to find the drug name by code
drug_code_dic = {}
for key, value in drug_dic.items():
    drug_code_dic[value[-1]]=[key, value[0]]
pickle.dump(drug_code_dic, open('../Data/parsed/drug_code_dic.pk', 'wb'))
list(drug_code_dic.items())[:10]

[(4, ['etanercept', 'DB00005']),
 (334, ['zoledronic acid', 'DB00399']),
 (931, ['atorvastatin', 'DB01076']),
 (274, ['metformin', 'DB00331']),
 (343, ['rosiglitazone', 'DB00412']),
 (174, ['glimepiride', 'DB00222']),
 (305, ['levonorgestrel', 'DB00367']),
 (1077, ['epoprostenol', 'DB01240']),
 (1060, ['budesonide', 'DB01222']),
 (626, ['imiquimod', 'DB00724'])]

## Loading all reprots

In [18]:
all_pd = pickle.load(open('../Data/curated/patient_safety.pk', 'rb'))  

all_pd.head(3)

,version,report_id,case_id,country,qualify,serious,s1,s2,s3,s4,s5,s6,receivedate,receiptdate,age,gender,weight,SE,drugs,indications,lastingdays,date,receipt_date
0,18,6161368,PHEH2006US13340,US,5,1,1,0,1,0,0,1,2498,4776,68,2,0,"[10001488, 10002034, 10002855, 10002948, 10003...",[],[],2278,2006-11-03,2013-01-28
1,2,7820167,US-ROCHE-735882,US,4,1,0,0,1,0,0,1,4069,4764,39,2,68,"[10000081, 10003246, 10009900, 10012378, 10013...",[],[],695,2011-02-21,2013-01-16
2,11,7683841,PHHY2009CA17701,CA,3,1,0,0,1,0,0,1,3982,4777,-1,2,0,"[10000081, 10000087, 10001949, 10003445, 10006...",[],[],795,2010-11-26,2013-01-29


## Narrow down based on country and qualification

In [19]:
all_pd_US = all_pd[all_pd.country=='US']
print('Focus on US, reports #', all_pd_US.shape)

id_qua = [i in ['1', '2', '3'] for i in all_pd_US.qualify ]
all_pd_US_pro = all_pd_US[id_qua]  # professional: 1,2,3
print('Focus on professional qualification, reports #', all_pd_US_pro.shape)

pickle.dump(all_pd_US_pro, open('../Data/pandemic/all_pd_US_pro.pk', 'wb'))

Focus on US, reports # (6351817, 23)
Focus on professional qualification, reports # (2551071, 23)


# Generate data for all patients

The overall population (all patients) is denoted by 'uncondition'. Variables with 'uncondition' in name refers to the population of all patients.

## Generate data

In [20]:
all_pd_US_pro = pickle.load(open('../Data/pandemic/all_pd_US_pro.pk', 'rb'))  

In [21]:
# SE_list = list(SE_dic_df.code)
DF = all_pd_US_pro
SE_list = list(sorted(set(list(itertools.chain(*DF.SE)))))
print('#-SE in US pro',len(SE_list))

ind = [i in SE_list for i in meddra_pd_all.PT]
whatever_ = meddra_pd_all[ind]
print(whatever_.shape)
whatever_.head(3)

#-SE in US pro 15454
(15454, 9)


,PT,PT_name,HLT,HLT_name,HLGT,HLGT_name,SOC,SOC_name,SOC_abbr
0,10000014,17-alpha-hydroxylase deficiency,10021608,inborn errors of steroid synthesis,10027424,metabolic and nutritional disorders congenital,10010331,"congenital, familial and genetic disorders",Cong
0,10000029,5-alpha-reductase deficiency,10021608,inborn errors of steroid synthesis,10027424,metabolic and nutritional disorders congenital,10010331,"congenital, familial and genetic disorders",Cong
0,10000044,abdomen crushing,10083613,abdominal and gastrointestinal injuries nec,10022114,injuries nec,10022117,"injury, poisoning and procedural complications",Inj&P


In [22]:
# reports in 2020 
yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
n_re = []
for yr in yr_list:
    st = str(yr)+'-03-10'
    end = str(yr)+'-09-31'      
    ind = [st<i<end for i in all_pd_US_pro['receipt_date']]  # all ['date'] --> ['receipt_date']
    locals()['all_pd_US_pro_'+str(yr)]= all_pd_US_pro[ind]
    n_reports = len(locals()['all_pd_US_pro_'+str(yr)])
    n_re.append(n_reports)
    print('rows in {}:{}'.format(yr,n_reports))

rows in 2013:89334
rows in 2014:97804
rows in 2015:179383
rows in 2016:169233
rows in 2017:210736
rows in 2018:244005
rows in 2019:220920
rows in 2020:211152


In [23]:
yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

"""initialize the Data frame """
se_matrix = pd.DataFrame({'SE': list(whatever_.PT), 'name':list(whatever_['PT_name']) }) #'medra_ID': list(SE_dic_df['medra_ID'])

for yr in yr_list:
    n_report = len(locals()['all_pd_US_pro_'+str(yr)])
    print('{} year has {} reports'.format(yr, n_report))
    A =[]

    for se in tqdm(SE_list):
        name = locals()['all_pd_US_pro_'+str(yr)]
        indx = [se in j for j in name.SE]
        n_A = sum(indx)
        A.append(n_A)
    B = [n_report - i for i in A]
    se_matrix[str(yr)+'_A'] = A
    se_matrix[str(yr)+'_B'] = B

  0%|          | 9/15454 [00:00<02:58, 86.76it/s]

2013 year has 89334 reports


  0%|          | 9/15454 [00:00<03:06, 82.72it/s]

2014 year has 97804 reports


  0%|          | 5/15454 [00:00<05:15, 48.90it/s]

...

  0%|          | 5/15454 [00:00<06:17, 40.87it/s]

2019 year has 220920 reports


  0%|          | 5/15454 [00:00<06:10, 41.71it/s]

2020 year has 211152 reports


100%|██████████| 15454/15454 [06:11<00:00, 41.65it/s]


In [24]:
pickle.dump(se_matrix, open('../Data/pandemic/SE_uncondition_raw.pk', 'wb'))

In [25]:
para = se_matrix
yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
for yr in yr_list:  # calculate ROR  
    para[str(yr)+'_ROR'] = (para['2020_A']*para[str(yr)+'_B'])/(para['2020_B']*para[str(yr)+'_A'])

for yr in yr_list:  # calculate Delta: average difference  
    para[str(yr)+'_Delta'] = (para['2020_A'] - para[str(yr)+'_A'])/para[str(yr)+'_A']

pd.set_option('display.max_columns', None)
"""Note: 0/0 = NaN, 1/0 = inf"""
para.head()

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
0,10000014,17-alpha-hydroxylase deficiency,0,89334,0,97804,1,179382,0,169233,0,210736,0,244005,0,220920,0,211152,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN
1,10000029,5-alpha-reductase deficiency,0,89334,0,97804,0,179383,0,169233,0,210736,0,244005,0,220920,1,211151,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,10000044,abdomen crushing,0,89334,0,97804,0,179383,0,169233,0,210736,0,244005,0,220920,1,211151,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
3,10000050,abdominal adhesions,14,89320,14,97790,16,179367,7,169226,21,210715,16,243989,15,220905,12,211140,0.362603,0.396988,0.637138,1.373978,0.570278,0.866684,0.836999,-0.142857,-0.142857,-0.250000,0.714286,-0.428571,-0.250000,-0.200000
4,10000059,abdominal discomfort,559,88775,613,97191,1402,177981,1446,167787,1999,208737,2400,241605,2145,218775,1993,209159,1.513246,1.510763,1.209641,1.105658,0.994987,0.959236,0.971854,2.565295,2.251223,0.421541,0.378285,-0.003002,-0.169583,-0.070862


In [26]:
pickle.dump(para, open('../Data/pandemic/SE_uncondition.pk', 'wb'))  # update the dataframe with ROR and Delta

### Sanity check

In [27]:
uncondition_2019_history = pickle.load(open('../Data/pandemic/SE_uncondition.pk', 'rb')) 

In [28]:
uncondition_2019_history[uncondition_2019_history.name=='cardiac arrest']

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
1123,10007515,cardiac arrest,525,88809,750,97054,637,178746,867,168366,714,210022,667,243338,725,220195,1331,209821,1.073067,0.820883,1.780025,1.231869,1.865931,2.314266,1.926631,1.535238,0.774667,1.089482,0.535179,0.864146,0.995502,0.835862


In [29]:
x = uncondition_2019_history[uncondition_2019_history['2019_Delta']>0]
print(x.shape)
y = x[x['2020_A']>1000]
print(y.shape)

y[['SE', 'name','2018_A', '2019_A', '2020_A', '2019_Delta']]

(5724, 32)
(78, 32)


,SE,name,2018_A,2019_A,2020_A,2019_Delta
169,10001551,alanine aminotransferase increased,389,436,1222,1.802752
246,10002034,anaemia,1406,1482,1629,0.099190
355,10002855,anxiety,1781,1777,2113,0.189083
433,10003239,arthralgia,4331,3266,3744,0.146356
470,10003549,asthenia,3164,2957,3074,0.039567
534,10003988,back pain,3119,2121,2322,0.094767
903,10005750,blood pressure increased,1199,1020,1095,0.073529
1123,10007515,cardiac arrest,667,725,1331,0.835862
1237,10008190,cerebrovascular accident,1050,1098,1197,0.090164
1273,10008469,chest discomfort,876,826,1005,0.216707


In [30]:
uncondition_2019_history[uncondition_2019_history.name=='cough']

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
1601,10011224,cough,1113,88221,1269,96535,2474,176909,2140,167093,2503,208233,2636,241369,2451,218469,2874,208278,1.093755,1.049703,0.986719,1.077427,1.147974,1.263512,1.229957,1.58221,1.264775,0.161681,0.342991,0.148222,0.090288,0.172583


## Population distribution

In [31]:
all_pd_US_pro = pickle.load(open('../Data/pandemic/all_pd_US_pro.pk', 'rb'))  # update the dataframe with ROR and Delta
len(all_pd_US_pro)

2551071

In [32]:
st = '-03-10'
end = '-09-31'
ind = [st<i[4:]<end for i in all_pd_US_pro['receipt_date']]
all_pd_US_pro_period= all_pd_US_pro[ind]
n_all = len(all_pd_US_pro_period)
print('the #-reports during March 11-Sept 30, accmulated from 2013-2020', n_all)

n_male = len(all_pd_US_pro_period[all_pd_US_pro_period.gender=='1'])
n_female = len(all_pd_US_pro_period[all_pd_US_pro_period.gender=='2'])

in_young = [str(0)<str(i)<str(20) for i in all_pd_US_pro_period.age]

in_adult = [str(19)<str(i)<str(65) for i in all_pd_US_pro_period.age]

in_elderly = [str(64)<str(i) for i in all_pd_US_pro_period.age]
n_young = len(all_pd_US_pro_period[in_young])
n_adult = len(all_pd_US_pro_period[in_adult])
n_elderly = len(all_pd_US_pro_period[in_elderly])

# unknown sex: 
n_unknownsex = n_all-n_male-n_female
n_unknownage = n_all - n_young -n_adult-n_elderly
print('#-male reports',n_male, n_male/n_all)
print('#-female reports',n_female, n_female/n_all)
print('unknown sex: ', n_unknownsex, n_unknownsex/n_all)

# unknown age
print('#-young reports', n_young, n_young/n_all)

print('#-adult reports', n_adult, n_adult/n_all)

print('#-elderly reports',n_elderly, n_elderly/n_all)
print('unknown age', n_unknownage, n_unknownage/n_all)

the #-reports during March 11-Sept 30, accmulated from 2013-2020 1425371
#-male reports 484649 0.3400160379297741
#-female reports 784230 0.5501935987192107
unknown sex:  156492 0.10979036335101527
#-young reports 35987 0.025247461888869633
#-adult reports 508983 0.357088084435561
#-elderly reports 305685 0.21445995463637185
unknown age 574716 0.40320449903919753


In [35]:
## mean and std average
young_age = np.array(list(all_pd_US_pro_period[in_young].age))
print(young_age.mean(), young_age.std())

adult_age = np.array(list(all_pd_US_pro_period[in_adult].age))
print(adult_age.mean(), adult_age.std())

elderly_age = np.array(list(all_pd_US_pro_period[in_elderly].age))
print(elderly_age.mean(), elderly_age.std())

13.988801511656987 9.009386593710554
47.78402422084824 13.198427571170923
72.98214501856486 11.0588864576358


In [3]:
# "1= Physician
# 2= Pharmacist
# 3= Other Health Professional
# 4= Lawyer
# 5= Consumer"

## Qualification/reporter distribution
all_pd = pickle.load(open('../Data/curated/patient_safety.pk', 'rb'))  
print('#-of all reports',all_pd.shape)

all_pd_US = all_pd[all_pd.country=='US']
print('Focus on US, reports #', all_pd_US.shape)

st = '-03-10'
end = '-09-31'
ind = [st<i[4:]<end for i in all_pd_US['receipt_date']]
all_pd_US_period= all_pd_US[ind]

all_pd_US_period['qualify'].value_counts()

#-of all reports (9325731, 23)
Focus on US, reports # (6351817, 23)


5    2045491
1     594787
3     548261
2     282323
6     124447
4     113744
0        478
Name: qualify, dtype: int64

# Conditioned on Gender

## Male

In [5]:
all_pd_US_pro = pickle.load(open('../Data/pandemic/all_pd_US_pro.pk', 'rb'))  # update the dataframe with ROR and Delta
print(len(all_pd_US_pro))

all_pd_US_pro_male = all_pd_US_pro[all_pd_US_pro.gender=='1']

DF = all_pd_US_pro_male
# reports in 2020 
"""initialize the Data frame """
# SE_list = list(SE_dic_df.code)
# male_matrix = pd.DataFrame({'SE': SE_list, 'name':list(SE_dic_df.index), 'medra_ID': list(SE_dic_df['medra_ID'])})

SE_list = list(sorted(set(list(itertools.chain(*DF.SE)))))
ind = [i in SE_list for i in meddra_pd_all.PT]
whatever_ = meddra_pd_all[ind]
male_matrix = pd.DataFrame({'SE': list(whatever_.PT), 'name':list(whatever_['PT_name']) }) 


yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for yr in yr_list:
    st = str(yr)+'-03-10'
    end = str(yr)+'-09-31'
    ind = [st<i<end for i in DF['receipt_date']]
    locals()['all_pd_US_pro_'+str(yr)]= DF[ind]
    print('rows in {}:{}'.format(yr,len(locals()['all_pd_US_pro_'+str(yr)])))

    n_report = len(locals()['all_pd_US_pro_'+str(yr)])
    print('{} year has {} reports'.format(yr, n_report))
    A =[]
    for se in tqdm(SE_list):
        name = locals()['all_pd_US_pro_'+str(yr)]
        indx = [se in j for j in name.SE]
        n_A = sum(indx)
        A.append(n_A)
    B = [n_report - i for i in A]
    male_matrix[str(yr)+'_A'] = A
    male_matrix[str(yr)+'_B'] = B

2551071


  0%|          | 27/12439 [00:00<00:47, 262.17it/s]

rows in 2013:30468
2013 year has 30468 reports


  0%|          | 24/12439 [00:00<00:52, 238.62it/s]

rows in 2014:33824
2014 year has 33824 reports


...

  0%|          | 10/12439 [00:00<02:07, 97.17it/s]

rows in 2019:73675
2019 year has 73675 reports


  0%|          | 11/12439 [00:00<02:03, 100.80it/s]

rows in 2020:72733
2020 year has 72733 reports


100%|██████████| 12439/12439 [02:05<00:00, 99.03it/s] 


In [6]:
male_matrix.head(3)

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B
0,10000029,5-alpha-reductase deficiency,0,30468,0,33824,0,63073,0,58998,0,71237,0,79623,0,73675,1,72732
1,10000044,abdomen crushing,0,30468,0,33824,0,63073,0,58998,0,71237,0,79623,0,73675,1,72732
2,10000050,abdominal adhesions,5,30463,1,33823,7,63066,3,58995,9,71228,4,79619,4,73671,7,72726


In [7]:
pickle.dump(male_matrix, open('../Data/pandemic/SE_male_raw.pk', 'wb'))

In [8]:
para_male = male_matrix
yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
for yr in yr_list:  # calculate ROR  
    para_male[str(yr)+'_ROR'] = (para_male['2020_A']*para_male[str(yr)+'_B'])/(para_male['2020_B']*para_male[str(yr)+'_A'])

for yr in yr_list:  # calculate Delta: average difference  
    para_male[str(yr)+'_Delta'] = (para_male['2020_A'] - para_male[str(yr)+'_A'])/para_male[str(yr)+'_A']

pd.set_option('display.max_columns', None)
"""Note: 0/0 = NaN, 1/0 = inf"""
para_male.head()

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
0,10000029,5-alpha-reductase deficiency,0,30468,0,33824,0,63073,0,58998,0,71237,0,79623,0,73675,1,72732,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
1,10000044,abdomen crushing,0,30468,0,33824,0,63073,0,58998,0,71237,0,79623,0,73675,1,72732,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf,inf
2,10000050,abdominal adhesions,5,30463,1,33823,7,63066,3,58995,9,71228,4,79619,4,73671,7,72726,0.586423,3.255521,0.867173,1.892789,0.761757,1.915866,1.772739,0.400000,6.000000,0.000000,1.333333,-0.222222,0.750000,0.750000
3,10000059,abdominal discomfort,188,30280,176,33648,444,62629,482,58516,653,70584,806,78817,674,73001,582,72151,1.299208,1.542152,1.137819,0.979283,0.871914,0.788797,0.873674,2.095745,2.306818,0.310811,0.207469,-0.108729,-0.277916,-0.136499
4,10000060,abdominal distension,120,30348,150,33674,234,62839,191,58807,280,70957,393,79230,304,73371,265,72468,0.924801,0.820924,0.982003,1.125888,0.926695,0.737219,0.882573,1.208333,0.766667,0.132479,0.387435,-0.053571,-0.325700,-0.128289


In [9]:
pickle.dump(para_male, open('../Data/pandemic/SE_male.pk', 'wb'))  # update the dataframe with ROR and Delta

## Female

In [10]:
all_pd_US_pro_female = all_pd_US_pro[all_pd_US_pro.gender=='2']

DF = all_pd_US_pro_female
# reports in 2020 
"""initialize the Data frame """
# SE_list = list(SE_dic_df.code)
# female_matrix = pd.DataFrame({'SE': SE_list, 'name':list(SE_dic_df.index), 'medra_ID': list(SE_dic_df['medra_ID'])})
SE_list = list(sorted(set(list(itertools.chain(*DF.SE)))))
ind = [i in SE_list for i in meddra_pd_all.PT]
whatever_ = meddra_pd_all[ind]
female_matrix = pd.DataFrame({'SE': list(whatever_.PT), 'name':list(whatever_['PT_name']) }) 

yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
for yr in yr_list:
    st = str(yr)+'-03-10'
    end = str(yr)+'-09-31'
    ind = [st<i<end for i in DF['receipt_date']]
    locals()['all_pd_US_pro_'+str(yr)]= DF[ind]
    print('rows in {}:{}'.format(yr,len(locals()['all_pd_US_pro_'+str(yr)])))

    n_report = len(locals()['all_pd_US_pro_'+str(yr)])
    print('{} year has {} reports'.format(yr, n_report))
    A =[]
    for se in tqdm(SE_list):
        name = locals()['all_pd_US_pro_'+str(yr)]
        indx = [se in j for j in name.SE]
        n_A = sum(indx)
        A.append(n_A)
    B = [n_report - i for i in A]
    female_matrix[str(yr)+'_A'] = A
    female_matrix[str(yr)+'_B'] = B

  0%|          | 15/13433 [00:00<01:31, 147.32it/s]

rows in 2013:50468
2013 year has 50468 reports


  0%|          | 15/13433 [00:00<01:33, 143.87it/s]

rows in 2014:55218
2014 year has 55218 reports


  0%|          | 8/13433 [00:00<02:49, 79.31it/s]

...

  0%|          | 7/13433 [00:00<03:18, 67.61it/s]

rows in 2019:117965
2019 year has 117965 reports


  0%|          | 7/13433 [00:00<03:26, 65.08it/s]

rows in 2020:113907
2020 year has 113907 reports


100%|██████████| 13433/13433 [03:14<00:00, 69.01it/s]


In [11]:
pickle.dump(female_matrix, open('../Data/pandemic/SE_female_raw.pk', 'wb'))

In [12]:
para_female = female_matrix
yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
for yr in yr_list:  # calculate ROR  
    para_female[str(yr)+'_ROR'] = (para_female['2020_A']*para_female[str(yr)+'_B'])/(para_female['2020_B']*para_female[str(yr)+'_A'])

for yr in yr_list:  # calculate Delta: average difference  
    para_female[str(yr)+'_Delta'] = (para_female['2020_A'] - para_female[str(yr)+'_A'])/para_female[str(yr)+'_A']

pd.set_option('display.max_columns', None)
"""Note: 0/0 = NaN, 1/0 = inf"""
para_female.head()

,SE,name,2013_A,2013_B,2014_A,2014_B,2015_A,2015_B,2016_A,2016_B,2017_A,2017_B,2018_A,2018_B,2019_A,2019_B,2020_A,2020_B,2013_ROR,2014_ROR,2015_ROR,2016_ROR,2017_ROR,2018_ROR,2019_ROR,2013_Delta,2014_Delta,2015_Delta,2016_Delta,2017_Delta,2018_Delta,2019_Delta
0,10000044,abdomen crushing,0,50468,0,55218,0,97402,0,91947,0,114661,0,141138,0,117965,0,113907,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10000050,abdominal adhesions,9,50459,13,55205,9,97393,3,91944,12,114649,10,141128,10,117955,5,113902,0.246113,0.186412,0.475033,1.345367,0.419399,0.619515,0.517792,-0.444444,-0.615385,-0.444444,0.666667,-0.583333,-0.500000,-0.500000
2,10000059,abdominal discomfort,344,50124,410,54808,881,96521,891,91056,1226,113435,1469,139669,1298,116667,1272,112635,1.645512,1.509642,1.237256,1.154103,1.044889,1.073722,1.015049,2.697674,2.102439,0.443814,0.427609,0.037520,-0.134105,-0.020031
3,10000060,abdominal distension,237,50231,262,54956,500,96902,391,91556,562,114099,618,140520,564,117401,593,113314,1.109161,1.097703,1.014224,1.225409,1.062470,1.189928,1.089341,1.502110,1.263359,0.186000,0.516624,0.055160,-0.040453,0.051418
4,10000077,abdominal mass,6,50462,11,55207,8,97394,11,91936,7,114654,9,141129,15,117950,10,113897,0.738416,0.440645,1.068882,0.733805,1.438066,1.376770,0.690390,0.666667,-0.090909,0.250000,-0.090909,0.428571,0.111111,-0.333333


In [13]:
pickle.dump(para_female, open('../Data/pandemic/SE_female.pk', 'wb'))  # update the dataframe with ROR and Delta

# Conditioned on Age

Based on [WHO](https://www.who.int/hiv/pub/guidelines/arv2013/intro/keyterms/en/) and the [Men Ageing And Health](https://apps.who.int/iris/bitstream/handle/10665/66941/WHO_NMH_NPH_01.2.pdf;jsessionid=A48157B9B4DFAA3A9874176D8A7C2894?sequence=1), the age group:

- Young: 1-19 
- Adult: 20-65
- Elderly: >65


## Bin age into groups

In [14]:
age_US_df = pickle.load(open('../Data/pandemic/all_pd_US_pro.pk', 'rb'))  # update the dataframe with ROR and Delta

# Bin age into groups 
age_US_df['age'] = pd.to_numeric(age_US_df['age'], errors='coerce')
bins = [1, 20, 65, max(age_US_df.age)+1] 
age_labels = ['young', 'adult','elderly']
age_US_df['age_group'] = pd.cut(age_US_df.age, bins, right = False, labels= age_labels)

## Three age groups

In [15]:
for age in age_labels:
    age_US_df_group = age_US_df[age_US_df.age_group==age]

    DF = age_US_df_group
    # reports in 2020 
    """initialize the Data frame """
#     SE_list = list(SE_dic_df.code)
#     age_matrix = pd.DataFrame({'SE': SE_list, 'name':list(SE_dic_df.index), 'medra_ID': list(SE_dic_df['medra_ID'])})
    """Remember sort the SE set!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!11"""
    SE_list = list(sorted(set(list(itertools.chain(*DF.SE)))))
    ind = [i in SE_list for i in meddra_pd_all.PT]
    whatever_ = meddra_pd_all[ind]
    age_matrix = pd.DataFrame({'SE': list(whatever_.PT), 'name':list(whatever_['PT_name']) }) 
    
    
    yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]
    for yr in yr_list:
        st = str(yr)+'-03-10'
        end = str(yr)+'-09-31'
        ind = [st<i<end for i in DF['receipt_date']]
        locals()['all_pd_US_pro_'+str(yr)]= DF[ind]
        print('rows in {}:{}'.format(yr,len(locals()['all_pd_US_pro_'+str(yr)])))

        n_report = len(locals()['all_pd_US_pro_'+str(yr)])
        print('{} year has {} reports'.format(yr, n_report))
        A =[] 
        for se in tqdm(SE_list):
            name = locals()['all_pd_US_pro_'+str(yr)]
            indx = [se in j for j in name.SE]
            n_A = sum(indx)
            A.append(n_A)
        B = [n_report - i for i in A]
        age_matrix[str(yr)+'_A'] = A
        age_matrix[str(yr)+'_B'] = B

    pickle.dump(age_matrix, open('../Data/pandemic/SE_'+age+'_raw.pk', 'wb'))

    para_age = age_matrix
    yr_list = [2013, 2014, 2015, 2016, 2017, 2018, 2019]
    for yr in yr_list:  # calculate ROR  
        para_age[str(yr)+'_ROR'] = (para_age['2020_A']*para_age[str(yr)+'_B'])/(para_age['2020_B']*para_age[str(yr)+'_A'])

    for yr in yr_list:  # calculate Delta: average difference  
        para_age[str(yr)+'_Delta'] = (para_age['2020_A'] - para_age[str(yr)+'_A'])/para_age[str(yr)+'_A']

    
    """Note: 0/0 = NaN, 1/0 = inf"""

    pickle.dump(para_age, open('../Data/pandemic/SE_'+age+'.pk', 'wb'))  # update the dataframe with ROR and Delta
    print(age,'related data saved')

  3%|▎         | 218/7105 [00:00<00:03, 2176.29it/s]

rows in 2013:3567
2013 year has 3567 reports


  3%|▎         | 200/7105 [00:00<00:03, 1993.12it/s]

rows in 2014:4091
2014 year has 4091 reports


  2%|▏         | 154/7105 [00:00<00:04, 1538.94it/s]

...

rows in 2019:45917
2019 year has 45917 reports


  0%|          | 13/10283 [00:00<01:20, 127.85it/s]

rows in 2020:46371
2020 year has 46371 reports


100%|██████████| 10283/10283 [01:19<00:00, 128.62it/s]

elderly related data saved


In [16]:
locals()['all_pd_US_pro_'+str(yr)].head(10)

,version,report_id,case_id,country,qualify,serious,s1,s2,s3,s4,s5,s6,receivedate,receiptdate,age,gender,weight,SE,drugs,indications,lastingdays,date,receipt_date,age_group
7667816,3,10058132,US-PFIZER INC-2014093363,US,1,2,0,0,0,0,0,0,5207,7013,74,1,0,"[10013700, 10028813]",[DB00482],[10033371],1806,2014-04-04,2019-03-15,elderly
7667849,2,15531775,US-BRISTOL-MYERS SQUIBB COMPANY-BMS-2018-095646,US,1,2,0,0,0,0,0,0,6866,7018,67,1,69.39,"[10060933, 10076476]","[DB00472, DB00572, DB00572, DB00996, DB01039, ...","[10030155, 10070592, 10070592, 10070592, 10070...",152,2018-10-19,2019-03-20,elderly
7667908,6,15505898,US-PFIZER INC-2018414114,US,3,2,0,0,0,0,0,0,6863,7019,70,2,0,[10013709],"[DB00563, DB00563, DB00563]","[10039073, 10039073, 10039073]",156,2018-10-16,2019-03-21,elderly
7667948,4,15478711,US-PFIZER INC-2018399211,US,1,2,0,0,0,0,0,0,6856,7027,65,2,99,"[10005481, 10018356, 10038428]","[DB09073, DB09073]","[10035226, 10055113]",171,2018-10-09,2019-03-29,elderly
7668003,2,15545327,US-PFIZER INC-2018433974,US,3,2,0,0,0,0,0,0,6871,7016,73,2,0,[10013700],"[DB00318, DB00533, DB01076]","[10037440, 10037440, 10037440]",145,2018-10-24,2019-03-18,elderly
7668074,3,15548251,"US-ALNYLAM PHARMACEUTICALS, INC.-ALN-2018-000218",US,1,1,0,0,1,0,0,1,6871,7017,75,2,71.4,[10007882],"[DB00162, DB00162, DB00181, DB00193, DB00211, ...","[10003553, 10003553, 10003553, 10003553, 10003...",146,2018-10-24,2019-03-19,elderly
7668086,2,15540366,US-PFIZER INC-2018429645,US,3,2,0,0,0,0,0,0,6870,7017,66,1,0,[10013709],[DB00230],[10029331],147,2018-10-23,2019-03-19,elderly
7668169,4,13010086,US-CELGENEUS-USA-2016104725,US,1,1,0,0,0,0,0,1,6186,7026,71,2,58.11,"[10016256, 10034754, 10035528, 10037844, 10061...","[DB00480, DB00480, DB00480, DB00480]","[10028533, 10028533, 10070592, 10070592]",840,2016-12-08,2019-03-28,elderly
7668172,5,14285441,US-PFIZER INC-2017532858,US,1,2,0,0,0,0,0,0,6557,7018,69,2,0,"[10010264, 10022437, 10040851, 10076309]","[DB00175, DB00222, DB00230, DB00230, DB00331, ...","[10005423, 10012601, 10012601, 10012680, 10020...",461,2017-12-14,2019-03-20,elderly
7668189,2,15598934,US-PFIZER INC-2018454583,US,3,2,0,0,0,0,0,0,6886,7010,70,2,0,"[10013700, 10013968, 10019211, 10027175]",[DB00295],[10072005],124,2018-11-08,2019-03-12,elderly
